In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Streaming WordCount") \
        .master("local[3]") \
        .config("spark.streaming.stopGracefullyOnShutdown", "true") \
        .config("spark.sql.shuffle.partitions", 3) \
        .getOrCreate()

In [3]:
spark

In [4]:
# Creamos un flujo de escucha sobre netcat en localhost:9999
lines_df = spark.readStream \
        .format("socket") \
        .option("host", "localhost") \
        .option("port", "9999") \
        .load()

In [6]:
# leemos las lineas y las pasamos a palabras. Sobre ellas, realizamos una agrupación count
from pyspark.sql.functions import explode, split
words_df = lines_df.select(explode(split(lines.value, ' ')).alias('palabra')))
counts_df = words_df.groupBy("palabra").count()

In [7]:
word_count_query = counts_df.writeStream \
        .format("console") \
        .outputMode("complete") \
        .option("checkpointLocation", "chk-point-dir") \
        .start()

In [8]:
word_count_query.awaitTermination()

StreamingQueryException: Connection refused (Connection refused)
=== Streaming Query ===
Identifier: [id = 8785c611-fecb-4550-ba1a-8176abba7a6e, runId = f735664a-f9cb-46a0-89c1-e25692466c04]
Current Committed Offsets: {}
Current Available Offsets: {TextSocketV2[host: localhost, port: 9999]: -1}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
WriteToMicroBatchDataSource ConsoleWriter[numRows=20, truncate=true]
+- Aggregate [word#3], [word#3, count(1) AS count#7L]
   +- Project [word#3]
      +- Generate explode(split(value#0,  , -1)), false, [word#3]
         +- StreamingDataSourceV2Relation [value#0], org.apache.spark.sql.execution.streaming.sources.TextSocketTable$$anon$1@531a78d4, TextSocketV2[host: localhost, port: 9999]
